# **LeNet**

## General Imports

In [1]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

## Choosing the device

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Running on GPU(cuda)")
else:
    device = torch.device("cpu")
    print("Running on CPU")

Running on GPU(cuda)


## The LeNet architecture

In [4]:
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=6,kernel_size=5,stride=1,padding=2)
        self.conv2 = nn.Conv2d(in_channels=6,out_channels=16,kernel_size=5,stride=1)
        self.conv3 = nn.Conv2d(in_channels=16,out_channels=120,kernel_size=5,stride=1)

        self.pool1 = nn.AvgPool2d(kernel_size=2,stride=2)
        self.pool2 = nn.AvgPool2d(kernel_size=2,stride=2)

        self.fc1 = nn.Linear(in_features=120,out_features=84)
        self.fc2 = nn.Linear(in_features=84,out_features=10)

    def forward(self,x):
        x = self.conv1(x)
        x = nn.Tanh()(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = nn.Tanh()(x)
        x = self.pool2(x)

        x = self.conv3(x)
        x = nn.Flatten()(x)
        x = nn.Tanh()(x)
        x = self.fc1(x)
        x = nn.Tanh()(x)
        x = self.fc2(x)

        output = torch.nn.functional.softmax(x,dim=1)
        return output

## Training and Testing functions

In [5]:
def train(model, device, train_loader, optimizer, epoch, print_step):
    for batch_idx, (img, label) in enumerate(train_loader):
        img, label = img.to(device), label.to(device)
        optimizer.zero_grad()
        output = model(img)
        
        loss = torch.nn.CrossEntropyLoss()(output, label)
        loss.backward()

        optimizer.step()
        if batch_idx % print_step == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(img), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [6]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for img, label in test_loader:
            img, label = img.to(device), label.to(device)
            output = model(img)
            test_loss += torch.nn.CrossEntropyLoss(reduction='sum')(output, label).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(label.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

## Hyperparameters

In [7]:
lr = 1e-2
momentum = 0.9
gamma = 0.9
epochs = 10
batch_size=64

## Loading the data

In [10]:
mnist_train = datasets.MNIST(root='../data',train=True,download=True, transform=transforms.ToTensor())
mnist_test = datasets.MNIST(root='../data',train=False,transform=transforms.ToTensor())

cifar100_train = datasets.CIFAR100(root='../data',train=True,download=True,transform=transforms.ToTensor())
cifar100_test = datasets.MNIST(root='../data',train=False,transform=transforms.ToTensor())

#imagenet_train = datasets.ImageNet(root='../data',train=True,download=True, transform=transforms.ToTensor())
#imagenet_test = datasets.ImageNet(root='../data',train=False,transform=transforms.ToTensor())

Files already downloaded and verified


## Training the model

In [9]:
model = LeNet5().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=gamma)

train_loader = DataLoader(mnist_train, batch_size=batch_size)
test_loader = DataLoader(mnist_test, batch_size=batch_size)

for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, epoch, 128)
        test(model, device, test_loader)
        scheduler.step()
torch.save(model.state_dict(), "LeNet5.pt")

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.302020
Train Epoch: 1 [8192/60000 (14%)]	Loss: 2.301078
Train Epoch: 1 [16384/60000 (27%)]	Loss: 2.300002
Train Epoch: 1 [24576/60000 (41%)]	Loss: 2.294223
Train Epoch: 1 [32768/60000 (55%)]	Loss: 2.268214
Train Epoch: 1 [40960/60000 (68%)]	Loss: 2.127529
Train Epoch: 1 [49152/60000 (82%)]	Loss: 1.933382
Train Epoch: 1 [57344/60000 (96%)]	Loss: 1.763652

Test set: Average loss: 1.7982, Accuracy: 7019/10000 (70.19%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 1.796290
Train Epoch: 2 [8192/60000 (14%)]	Loss: 1.770450
Train Epoch: 2 [16384/60000 (27%)]	Loss: 1.651168
Train Epoch: 2 [24576/60000 (41%)]	Loss: 1.694643
Train Epoch: 2 [32768/60000 (55%)]	Loss: 1.614069
Train Epoch: 2 [40960/60000 (68%)]	Loss: 1.593203
Train Epoch: 2 [49152/60000 (82%)]	Loss: 1.675602
Train Epoch: 2 [57344/60000 (96%)]	Loss: 1.540626

Test set: Average loss: 1.5657, Accuracy: 9149/10000 (91.49%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 1.576176
Train Epoch: 3 [8192/60000 (14%)]	Loss: